# XRD Analysis

## Imports
* libraries 
* from python file

In [1]:
import pandas as pd
import fabio
import pyFAI

import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State

from XRD_methods import *

## Parameters

In [2]:
miller_indices = {
    'FCC': (
        [1, 1, 1],
        [2, 0, 0],
        [2, 2, 0],
        [3, 1, 1],
        [2, 2, 2],
        [4, 0, 0],
        [3, 3, 1],
        [4, 2, 0],
        [4, 2, 2],
        [3, 3, 3],
    )
}

In [3]:
lamb1 = 0.3905 

In [4]:
a = 3.6150 

In [5]:
d0 = d_from_hkl(miller_indices['FCC'], a)

In [6]:
tth0 = tth_from_d(d0, lamb1)

## Files

In [7]:
poni = 'C:/Users/starlord/Desktop/Summer Research 2020/App/diFFit/qg/ceria_callibration_geometry.poni'

rolled_file = "C:/Users/starlord/Desktop/Summer Research 2020/App/diFFit/qg/images/copper_as_rolled_003.tiff"
annealed_file = "C:/Users/starlord/Desktop/Summer Research 2020/App/diFFit/qg/images/copper_annealed_002.tiff"
hammered_file = "C:/Users/starlord/Desktop/Summer Research 2020/App/diFFit/qg/images/copper_hammered_002.tiff"

In [8]:
rolled = element(rolled_file)
annealed = element(annealed_file)
hammered = element(hammered_file)

## Filter For Bad Pixels

In [9]:
lim = 100000

rolled.filt(lim)
annealed.filt(lim)
hammered.filt(lim)

## Integrate to find Relative Intensities

In [10]:
rolled.hist, rolled.polar = integrateHistAndPolar(rolled.data_filt, poni)
annealed.hist, annealed.polar = integrateHistAndPolar(annealed.data_filt, poni)
hammered.hist, hammered.polar = integrateHistAndPolar(hammered.data_filt, poni)

In [11]:
rolled.find_background(width=1000)

In [12]:
annealed.find_background(width=1000)

In [13]:
hammered.find_background(width=1000)

## Fit the Peaks of the Histogram to Gaussian Curves

In [16]:
rolled.ycomps, rolled.fit_values = fit_hist(tth0, rolled.hist['x'], rolled.hist['y']-rolled.bgr)

In [17]:
annealed.ycomps, annealed.fit_values = fit_hist(tth0, annealed.hist['x'], annealed.hist['y']- annealed.bgr)

In [18]:
hammered.ycomps, hammered.fit_values = fit_hist(tth0, hammered.hist['x'], hammered.hist['y']- hammered.bgr)

## Strain
$$
\frac{d-d_0}{d_0}
$$

$$
d = \frac{n \lambda}{2 \sin \theta}
$$

In [20]:
rolled.fit_values['d'] = d_from_tth(rolled.fit_values.center.values, lamb1)

In [21]:
annealed.fit_values['d'] = d_from_tth(annealed.fit_values.center.values, lamb1)

In [22]:
hammered.fit_values['d'] = d_from_tth(hammered.fit_values.center.values, lamb1)

In [23]:
rolled.fit_values['strain'] = (rolled.fit_values['d']-d0)/d0

In [24]:
annealed.fit_values['strain'] = (annealed.fit_values['d']-d0)/d0

In [25]:
hammered.fit_values['strain'] = (hammered.fit_values['d']-d0)/d0

## Generate The Figures 

In [26]:
rolled.gen_figs(tth0)

In [27]:
annealed.gen_figs(tth0)

In [28]:
hammered.gen_figs(tth0)

## Set up the html dashboard to view the results

if I want to have the elements snap from being next to each other and be stacked as the window resizes, need to put in html divs not rows.

In [29]:
app = dash.Dash(meta_tags=[{
    "name": "viewport",
    "content": "width=device-width, initial-scale=1"
}],
                external_stylesheets=[dbc.themes.LITERA])

In [30]:
navbar = dbc.Navbar([
    dbc.Row(
        [dbc.Col([dbc.NavbarBrand("XRD Analysis", className="navbar-brand")])])
],
                    color='primary',
                    className="navbar navbar-expand-lg navbar-dark bg-primary")

In [31]:
banner = html.Div([
    dbc.Row([
        dbc.Col([
            html.H5("Copper Diffraction"),
            html.H3("Welcome to the Copper Diffraction Dashboard"),
            html.Div(
                id='intro',
                children=
                "Compare and Contrast the effects of different processes on the latice structure of copper"
            )
        ],
                width={
                    "size": 10,
                    "offset": 1,
                },
                align='center')
    ])
],
                  id="banner")

In [32]:
def show_one(element):

    return dbc.Container([
        dbc.Row([
            dbc.Col([dcc.Graph(figure=element.hist_fig_2D)],
                    width={
                        "size": 7,
                    }),
            dbc.Col([
                html.Div([dcc.Graph(figure=element.hist_fig)]),
                html.Div([
                    dbc.Table.from_dataframe(element.fit_values.round(4).drop(
                        ["model", "amplitude"], axis=1),
                                             striped=False,
                                             bordered=False,
                                             className="table table-hover",
                                             size='sm',
                                             responsive=True),
                ]),
            ],
                    width={'size': 5}),
        ]),
    ],
                         
                         fluid="True",
                         style={
                             "margin": "20px",
                             "border": "2px solid black",
                         })

### Original Images

In [33]:
images = html.Div([
    dbc.Row([dbc.Col([dcc.Graph(figure=rolled.img_fig)], align='center')]),
    dbc.Row([dbc.Col([dcc.Graph(figure=annealed.img_fig)], align='center')]),
    dbc.Row([dbc.Col([dcc.Graph(figure=hammered.img_fig)], align='center')]),
])

### 2D Histograms

In [34]:
hists_2D = html.Div([
    dbc.Row([dbc.Col([dcc.Graph(figure=rolled.hist_fig_2D)], align='center')]),
    dbc.Row(
        [dbc.Col([dcc.Graph(figure=annealed.hist_fig_2D)], align='center')]),
    dbc.Row(
        [dbc.Col([dcc.Graph(figure=hammered.hist_fig_2D)], align='center')]),
])

## Tabs to switch between types of plots

In [35]:
tabs = dbc.Container([
    dbc.Tabs([
        dbc.Tab(label="Histograms", tab_id="hist"),
        dbc.Tab(label="Polar Plots", tab_id="polar"),
        dbc.Tab(label="2D Histograms", tab_id="2D_hist"),
        dbc.Tab(label="Images", tab_id="images"),
    ],
             id="tabs",
             active_tab="hist",
             className="nav nav-pills"),
    html.Div(id="tab_content"),
],fluid=True)

In [36]:
app.layout = html.Div(children=[navbar, banner, show_one(rolled), show_one(annealed), show_one(hammered)] )

In [37]:
@app.callback(Output("tab_content", "children"), [Input("tabs", "active_tab")])
def render_tab_content(active_tab):
    if active_tab is not None:
        if active_tab == "hist":
            return histograms
        elif active_tab == "polar":
            return polar_plots
        elif active_tab == "2D_hist":
            return hists_2D
        elif active_tab == "images":
            return images
    return html.H1(["No Tab Selected"])

# Run App locally 

In [38]:
if __name__ == '__main__':
    app.run_server(debug=False)  

Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



INFO:__main__: Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.



INFO:__main__: in production, use a production WSGI server like gunicorn instead.



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [15/Aug/2020 15:51:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Aug/2020 15:51:18] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Aug/2020 15:51:19] "GET /_favicon.ico?v=1.13.4 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Aug/2020 15:51:19] "GET /_dash-layout HTTP/1.1" 200 -
